In [ ]:
!pip install ipywidgets

In [ ]:
import os
import time
import pickle
import numpy as np
import pandas as pd
from numpy.linalg import norm
from tqdm import tqdm, tqdm_notebook
import keras.utils as image
from tensorflow.keras.applications import ResNet50
from keras.applications.vgg16 import preprocess_input

# Extracting Features from Images

In [ ]:
#Initialize Model
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#Define Function to extract Features
def extract_features(img_path, model):
    
    #Preprocessing Input Image
    input_shape = (224, 224, 3)
    img = image.load_img(img_path, target_size=(input_shape[0], input_shape[1])) #Reshape input image size into target size
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    
    #Getting features from the Image
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

****Explaination of the above code:****

1.  This loads an image and resizes the image to (224, 224): img = image.load_img(img_path, target_size=(224, 224))

2.  The img_to_array() function adds channels: x.shape = (224, 224, 3) for RGB and (224, 224, 1) for gray image:          x = image.img_to_array(img) 

3.  expand_dims() is used to add the number of images: x.shape = (1, 224, 224, 3): x = np.expand_dims(x, axis=0)

4.  preprocess_input subtracts the mean RGB channels of the imagenet dataset. This is because the model you are using has     been trained on a different dataset: x.shape is still (1, 224, 224, 3): x = preprocess_input(x)

* **numpy.linalg.norm**: This function returns one of the seven matrix norms or one of the infinite vector norms depending upon the value of its parameters.
* **preprocess_input**: The preprocess_input function is meant to adequate your image to the format the model requires.Some models use images with values ranging from 0 to 1. Others from -1 to +1. Others use the "caffe" style, that is not normalized, but is centered.

In [ ]:
#Find Images in the Root Directiry and making list of those Images
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']
def get_file_list(root_dir):
    file_list = []
    counter = 1
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
                counter += 1
    return file_list

In [ ]:
# path to the datasets
root_dir = './Caltech101/Caltech101/train'
filenames = sorted(get_file_list(root_dir))

In [ ]:
from tqdm.notebook import tqdm

feature_list = []
for i in tqdm(range(len(filenames))):
    feature_list.append(extract_features(filenames[i], model))


In [ ]:
#Making pickle file of filenames and features of each files for future references
pickle.dump(feature_list, open('./features-caltech101-resnet.pickle', 'wb'))
pickle.dump(filenames, open('./filenames-caltech101.pickle','wb'))

In [ ]:
#Getting filenames and features from pickle files
filenames = pickle.load(open('./filenames-caltech101.pickle', 'rb'))
feature_list = pickle.load(open('./features-caltech101-resnet.pickle', 'rb'))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('./Caltech101/Caltech101/train',
                                                   class_mode='categorical')

In [ ]:
print(train_generator.filenames[0])
print(train_generator.filenames)

In [ ]:
#Extracting image name and labels of each Image to make a dataframe that can be used later while using annoy

file_names = []
labels = []
for files in train_generator.filenames:
    file = files.split('\\')[1]
    label = files.split('\\')[0]
    file_names.append(file)
    labels.append(label)

In [ ]:
#Forming dataframe containing Image Id, Features of Images and Labels of the Images\
df = pd.DataFrame({'img_id':file_names, 'img_repr': feature_list, 'label': labels})

In [ ]:
df.head()

# Using Spotify's Annoy for Similarity Search

In [ ]:
len(df['img_repr'][0])

In [ ]:
!pip install annoy

In [ ]:
from annoy import AnnoyIndex
import random

f = len(df['img_repr'][0])
t = AnnoyIndex(f, metric='euclidean')

for i in tqdm(range(len(feature_list))):
    t.add_item(i, feature_list[i])
    
_ = t.build(150)

In [ ]:
import tensorflow as tf
image_path = './test.jpg'

# Load and preprocess the image
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
image = tf.keras.preprocessing.image.img_to_array(image)
image = tf.keras.applications.mobilenet_v2.preprocess_input(image)

# Expand dimensions to match the model's input shape
image = np.expand_dims(image, axis=0)
features = model.predict(image)
feature_vector = features.flatten()
print(len(feature_list))
t.unbuid()
t.add_item(6162, feature_vector)
_ = t.build(150)

****Explaination of the above code:****

* In this first part, we set the number of dimensions of the space of our net to number of features extracted from image i.e. **100352** and we choose to use an euclidean distance.
* Then, in the next step, we create a **6162** items i.e. equal to total number of images, each one with its correspondenly **100352** dimensional vector, which can be also understood as a point in a 100352th dimensional space.
* We build then the corresponding net by using 1000 trees.

In [ ]:
#Defining function to get similar images output in dataframe of the base image index we give as parameter

def get_similar_images_annoy(img_index):
    start = time.time()
    base_img_id, base_vector, base_label  = df.iloc[img_index, [0, 1, 2]]
    similar_img_ids = t.get_nns_by_item(img_index, 4)
    end = time.time()
    # Now we want to get the 5 elements sorted by its euclidean distance relative to the image indexed item
    print(f'{(end - start) * 1000} ms')
    return base_img_id, base_label, df.iloc[similar_img_ids[1:]]

In [ ]:
#Querying the AnnoyIndex    
base_image, base_label, similar_images_df = get_similar_images_annoy(2200)

In [ ]:
print('Base Image Id:', base_image)
print('Base Image Label:', base_label)

In [ ]:
#Dataframe of similar images
similar_images_df.head()

# Plotting Base and Similar Images

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 as cv

In [ ]:
def show_images(root_dir):
    plt.figure(figsize = (16,9))
    
    plt.subplot(1,4,1)
    path = os.path.join(root_dir, base_label, base_image)
    image = mpimg.imread(path)
    plt.imshow(image)
    plt.title('Base Image')
    plt.axis('off')
    
    for i in range(len(similar_images_df)):
        path = os.path.join(root_dir, similar_images_df.iloc[i,2],similar_images_df.iloc[i,0])
        image = mpimg.imread(path)
        plt.subplot(1,4,i+2)
        plt.imshow(image)
        plt.title('Similar Image')
        plt.axis('off')

In [ ]:
root_dir = './Caltech101/Caltech101/train/'
show_images(root_dir)